# deepseek-ai/deepseek-coder-6.7b-instruct LMI deployment guide
In this tutorial, you will use LMI container from DLC to SageMaker and run inference with it.

Please make sure the following permission granted before running the notebook:

- S3 bucket push access
- SageMaker access

## Step 1: Let's bump up SageMaker and import stuff

In [ ]:
%pip install sagemaker --upgrade  --quiet
%pip install boto3

In [ ]:
%pip show sagemaker

In [ ]:
import boto3
import sagemaker
from sagemaker import Model, image_uris, serializers, deserializers

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment
sagemaker_default_bucket = sess.default_bucket()

## Step 2: Start preparing model artifacts
In LMI contianer, we expect some artifacts to help setting up the model
- serving.properties (required): Defines the model server settings
- model.py (optional): A python file to define the core inference logic
- requirements.txt (optional): Any additional pip wheel need to install

In [ ]:
model_name="deepseek-ai/deepseek-coder-6.7b-instruct"
model_lineage=model_name.split("/")[0]
model_specific_name = model_name.split("/")[1]
cmd=f's/option.model_id=model_name/option.model_id={model_lineage}\/{model_specific_name}/g'
print(cmd)

### Step 2.1 Writing serving.properties

#### Option1(Global region)

- It's better to pre-download the model and upload it to S3, then use the s3url for deployment.

In [ ]:
with open("serving.properties", "w") as wf:
    wf.write("engine=Python\n")
    wf.write(f"option.model_id={model_name}\n")
    wf.write("option.rolling_batch=vllm\n")
    wf.write("option.max_model_len=8192\n")

#### Option2(China region)

1. Download model from Hugging face
2. Upload model to S3 Bucket
3. Write serving.properties using s3url

In [ ]:
!pip install -U huggingface_hub

In [ ]:
import os
from pathlib import Path

os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# local_model_path_name = "Qwen2-72B-Instruct"
local_model_path = Path(model_specific_name)
local_model_path.mkdir(exist_ok=True)

s3_model_prefix = f"lmi/{model_specific_name}"
s3url=f"s3://{sagemaker_default_bucket}/{s3_model_prefix}"

In [ ]:
!huggingface-cli download --resume-download {model_name} --local-dir {local_model_path}

In [ ]:
# Upload model to S3
!aws s3 cp {local_model_path} {s3url} --recursive

In [ ]:
with open("serving.properties", "w") as wf:
    wf.write("engine=Python\n")
    wf.write(f"option.model_id={s3url}\n")
    wf.write("option.rolling_batch=vllm\n")
    wf.write("option.max_model_len=8192\n")

### Step 2.2 Writing model.py for Message API (optional) 
- In some scenarios, we need to perform some customized processing for the input or output. We could use model.py to do customization.
- Here we use model.py to customize input formatter for better Message API format

[reference link](https://docs.djl.ai/docs/serving/serving/docs/lmi/user_guides/input_formatter_schema.html)

In [ ]:
%%writefile model.py
from djl_python.huggingface import HuggingFaceService
from djl_python.chat_completions.chat_utils import is_chat_completions_request
from djl_python.chat_completions.chat_properties import ChatProperties
from djl_python.inputs import Input
from djl_python.outputs import Output
from djl_python.encode_decode import encode, decode
import logging
import json
import types
import re

_service = HuggingFaceService()


def parse_chat_completions_request(inputs: map, is_rolling_batch: bool,
                                   tokenizer):
    if not hasattr(tokenizer, "apply_chat_template"):
        raise AttributeError(
            f"Cannot provide chat completion for tokenizer: {tokenizer.__class__}, "
            f"please ensure that your tokenizer supports chat templates.")
    chat_params = ChatProperties(**inputs)
    _param = chat_params.model_dump(by_alias=True, exclude_none=True)
    _messages = _param.pop("messages")
    _inputs = tokenizer.apply_chat_template(_messages, tokenize=False, add_generation_prompt=True)

    _param[
        "do_sample"] = chat_params.temperature is not None and chat_params.temperature > 0.0
    _param["details"] = True  # Enable details for chat completions
    _param[
        "output_formatter"] = "jsonlines_chat" if chat_params.stream else "json_chat"

    return _inputs, _param

def parse_inputs_params(input_map, item, input_format_configs):
    if is_chat_completions_request(input_map):
        _inputs, _param = parse_chat_completions_request(
            input_map, input_format_configs.is_rolling_batch,
            input_format_configs.tokenizer)
    else:
        _inputs = input_map.pop("inputs", input_map)
        _param = input_map.pop("parameters", {})

    if input_format_configs.is_rolling_batch:
        _param["stream"] = input_map.pop("stream", False)

    if "cached_prompt" in input_map:
        _param["cached_prompt"] = input_map.pop("cached_prompt")
    if "seed" not in _param:
        # set server provided seed if seed is not part of request
        if item.contains_key("seed"):
            _param["seed"] = item.get_as_string(key="seed")
    if not "output_formatter" in _param:
        _param["output_formatter"] = input_format_configs.output_formatter

    if isinstance(_inputs, list):
        return _inputs, _param, True
    else:
        return [_inputs], _param, False
    

def custom_input_formatter(self, inputs: Input, tokenizer, output_formatter) -> tuple[list[str], list[int], list[dict], dict, list]:

    input_data = []
    input_size = []
    parameters = []
    adapters = []
    errors = {}
    found_adapters = False
    batch = inputs.get_batches()
    # only for dynamic batch
    is_client_side_batch = [False for _ in range(len(batch))]
    for i, item in enumerate(batch):
        try:
            content_type = item.get_property("Content-Type")
            input_map = decode(item, content_type)
            _inputs, _param, is_client_side_batch[i] = parse_inputs_params(
                input_map, item, self.input_format_configs)

        except Exception as e:  # pylint: disable=broad-except
            logging.warning(f"Parse input failed: {i}")
            input_size.append(0)
            errors[i] = str(e)
            continue

        input_data.extend(_inputs)
        input_size.append(len(_inputs))

        for _ in range(input_size[i]):
            parameters.append(_param)

    return input_data,input_size, parameters, errors, batch


def handle(inputs: Input):
    """
    Default handler function
    """
    if not _service.initialized:
        # stateful model
        props = inputs.get_properties()
        print(f"props: {props}")
        # props['output_formatter'] = custom_output_formatter
        _service.initialize(props)
        _service.parse_input = types.MethodType(custom_input_formatter, _service)

    if inputs.is_empty():
        # initialization request
        return None

    return _service.inference(inputs)

### Compress model artifacts

In [ ]:
%%sh
mkdir mymodel
mv serving.properties model.py mymodel/
tar czvf mymodel.tar.gz mymodel/
rm -rf mymodel

## Step 3: Start building SageMaker endpoint
In this step, we will build SageMaker endpoint from scratch

### Getting the container image URI

[Large Model Inference available DLC](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#large-model-inference-containers)


In [ ]:
image_uri = image_uris.retrieve(
        framework="djl-lmi",
        region=sess.boto_session.region_name,
        version="v0.28.0"
    )

### Upload artifact on S3 and create SageMaker model

In [ ]:
s3_code_prefix = f"large-model-lmi/code-{model_lineage}-{model_specific_name}"
bucket = sess.default_bucket()  # bucket to house artifacts
code_artifact = sess.upload_data("mymodel.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {code_artifact}")

model = Model(image_uri=image_uri, model_data=code_artifact, role=role)

## Step4: Create SageMaker endpoint

You need to specify the instance to use and endpoint names

In [ ]:
instance_type = "ml.g5.2xlarge"
endpoint_name = sagemaker.utils.name_from_base(f"lmi-model-{model_lineage}-{model_specific_name.replace('.','-')}")

model.deploy(initial_instance_count=1,
             instance_type=instance_type,
             endpoint_name=endpoint_name,
             # container_startup_health_check_timeout=3600
            )

# our requests and responses will be in json format so we specify the serializer and the deserializer
predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
    serializer=serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer(),
)

## Step 5: Test and benchmark the inference

### Message API
Ref: https://docs.djl.ai/docs/serving/serving/docs/lmi/user_guides/chat_input_output_schema.html#message

In [ ]:
prompt = "帮我写一段快排代码"
# prompt = "你好"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

response = predictor.predict({
    "messages": messages,
    "max_tokens":250
    }
)

print(response["choices"][0]["message"]["content"])

### Standard schema
Ref: https://docs.djl.ai/docs/serving/serving/docs/lmi/user_guides/lmi_input_output_schema.html

In [ ]:
%pip install transformers -U

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
prompt = "帮我写一段快排代码"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    
)
parameters = {
        "max_new_tokens":250,
    }
response = predictor.predict(
    {"inputs": inputs, "parameters": parameters}
)

print(response["generated_text"])

### Streaming

In [ ]:
import json
import boto3

smr_client = boto3.client("sagemaker-runtime")

In [ ]:
import io
import json

class TokenIterator:
    def __init__(self, stream):
        self.byte_iterator = iter(stream)
        self.buffer = io.BytesIO()
        self.read_pos = 0

    def __iter__(self):
        return self

    def __next__(self):
        while True:
            self.buffer.seek(self.read_pos)
            line = self.buffer.readline()

            # print(line)
            if line and line[-1] == ord("\n"):
                self.read_pos += len(line)
                full_line = line[:-1].decode("utf-8")
                # print(full_line)
                line_data = json.loads(full_line.lstrip("data:").rstrip("/n"))
                return line_data["token"].get("text", "")
            chunk = next(self.byte_iterator)
            self.buffer.seek(0, io.SEEK_END)
            self.buffer.write(chunk["PayloadPart"]["Bytes"])

            
class MessageTokenIterator:
    def __init__(self, stream):
        self.byte_iterator = iter(stream)
        self.buffer = io.BytesIO()
        self.read_pos = 0

    def __iter__(self):
        return self

    def __next__(self):
        while True:
            self.buffer.seek(self.read_pos)
            line = self.buffer.readline()

            # print(line)
            if line and line[-1] == ord("\n"):
                self.read_pos += len(line)
                full_line = line[:-1].decode("utf-8")
                # print(full_line)
                line_data = json.loads(full_line.lstrip("data:").rstrip("/n"))
                return line_data["choices"][0]["delta"].get("content", "")
            chunk = next(self.byte_iterator)
            self.buffer.seek(0, io.SEEK_END)
            self.buffer.write(chunk["PayloadPart"]["Bytes"])

            
def get_realtime_response_stream(sagemaker_runtime, endpoint_name, payload):
    response_stream = sagemaker_runtime.invoke_endpoint_with_response_stream(
        EndpointName=endpoint_name,
        Body=json.dumps(payload),
        ContentType="application/json",
        CustomAttributes='accept_eula=false'
    )
    return response_stream


#### Standard schema streaming 

In [ ]:
prompt = "帮我写一段快排代码"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

parameters = {
        "max_new_tokens":256,
        "temperature": 0.7,
        "top_p": 0.8,
    }

payload = {
    "inputs":  inputs,
    "parameters": parameters,
    "stream": True ## <-- to have response stream.
}


response_stream = get_realtime_response_stream(smr_client, endpoint_name, payload)
# print_response_stream(response_stream)
for token in TokenIterator(response_stream["Body"]):
    # pass
    print(token, end="", flush=True)

#### Message Schema streaming 

In [ ]:
prompt = "帮我写一段快排代码"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
payload= {
    "messages": messages,
    "max_tokens":250,
    "temperature": 0.7,
    "top_p": 0.8,
    "stream": "true"
    }

response_stream = get_realtime_response_stream(smr_client, endpoint_name, payload)
for token in MessageTokenIterator(response_stream["Body"]):
    # pass
    print(token, end="", flash=True)

## Clean up the environment

In [ ]:
sess.delete_endpoint(endpoint_name)
sess.delete_endpoint_config(endpoint_name)
model.delete_model()